# Context Passing Between Tasks

Advanced context management for multi-agent workflows with structured task result passing.

In [ ]:
import sys
sys.path.append('../../../src')

from strands import Agent
from strands.models.ollama import OllamaModel

In [ ]:
model = OllamaModel(
    host="http://localhost:11434",
    model_id="granite4:tiny-h",
    temperature=0.1,
    top_p=0.9
)

In [ ]:
research_agent = Agent(
    model=model,
    system_prompt="You research and gather information on given topics."
)

analysis_agent = Agent(
    model=model,
    system_prompt="You analyze data and identify key insights."
)

summary_agent = Agent(
    model=model,
    system_prompt="You create concise summaries from detailed analysis."
)

recommendation_agent = Agent(
    model=model,
    system_prompt="You provide actionable recommendations based on analysis."
)

In [ ]:
tasks = {
    "market_research": {
        "description": "Research current market trends in renewable energy sector",
        "agent": research_agent,
        "dependencies": []
    },
    "competitor_analysis": {
        "description": "Analyze top 3 competitors in renewable energy market",
        "agent": analysis_agent,
        "dependencies": []
    },
    "market_analysis": {
        "description": "Analyze market opportunities and threats",
        "agent": analysis_agent,
        "dependencies": ["market_research", "competitor_analysis"]
    },
    "executive_summary": {
        "description": "Create executive summary of market analysis",
        "agent": summary_agent,
        "dependencies": ["market_analysis"]
    },
    "strategic_recommendations": {
        "description": "Provide strategic recommendations for market entry",
        "agent": recommendation_agent,
        "dependencies": ["market_analysis", "executive_summary"]
    }
}

In [ ]:
def build_task_context(task_id, tasks, results):
    """Build context from dependent tasks"""
    context = []
    for dep_id in tasks[task_id].get("dependencies", []):
        if dep_id in results:
            context.append(f"Results from {dep_id}: {results[dep_id]}")

    prompt = tasks[task_id]["description"]
    if context:
        prompt = "Previous task results:\n" + "\n\n".join(context) + "\n\nTask:\n" + prompt

    return prompt

In [ ]:
def execute_workflow_with_context(tasks, base_context=""):
    results = {}
    completed = set()
    
    while len(completed) < len(tasks):
        for task_id, task in tasks.items():
            if task_id in completed:
                continue
                
            if all(dep in completed for dep in task.get("dependencies", [])):
                print(f"Executing task: {task_id}")
                
                context = build_task_context(task_id, tasks, results)
                if base_context:
                    context = f"Base context: {base_context}\n\n{context}"
                
                result = task["agent"](context)
                results[task_id] = str(result)
                completed.add(task_id)
                
                print(f"Completed: {task_id}\n")
    
    return results

In [ ]:
base_context = """
Company: GreenTech Solutions
Industry: Renewable Energy Technology
Goal: Market entry strategy for solar panel manufacturing
Budget: $50M initial investment
Timeline: 18 months to market entry
"""

results = execute_workflow_with_context(tasks, base_context)

In [ ]:
for task_id, result in results.items():
    print(f"=== {task_id.upper().replace('_', ' ')} ===")
    print(result)
    print("-" * 80)
    print()

In [ ]:
print("WORKFLOW EXECUTION ORDER:")
execution_order = []
completed = set()

while len(completed) < len(tasks):
    for task_id, task in tasks.items():
        if task_id in completed:
            continue
        if all(dep in completed for dep in task.get("dependencies", [])):
            execution_order.append(task_id)
            completed.add(task_id)
            deps = task.get("dependencies", [])
            dep_str = f" (depends on: {', '.join(deps)})" if deps else " (no dependencies)"
            print(f"{len(execution_order)}. {task_id}{dep_str}")